In [2]:
# --- 1. Setup ---
!pip install tensorflow pandas scikit-learn gradio

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import gradio as gr


In [3]:
# --- 2. Dataset ---
texts = ["I love this product", "This is terrible", "Amazing experience", "Worst ever", "I am happy", "I hate it"]
labels = [1, 0, 1, 0, 1, 0]  # 1 = positive, 0 = negative

df = pd.DataFrame({"text": texts, "label": labels})


In [4]:
# --- 3. Preprocessing ---
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df['text'])
X = tokenizer.texts_to_sequences(df['text'])
X = pad_sequences(X, maxlen=10)
y = np.array(df['label'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [5]:
# --- 4. Model ---
model = Sequential([
    Embedding(input_dim=5000, output_dim=64, input_length=10),
    LSTM(64),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5, batch_size=2, validation_data=(X_test, y_test))


Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 365ms/step - accuracy: 0.3333 - loss: 0.6988 - val_accuracy: 0.5000 - val_loss: 0.6900
Epoch 2/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.5000 - loss: 0.6917 - val_accuracy: 0.5000 - val_loss: 0.6897
Epoch 3/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.5000 - loss: 0.6873 - val_accuracy: 0.5000 - val_loss: 0.6894
Epoch 4/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - accuracy: 0.6667 - loss: 0.6852 - val_accuracy: 0.5000 - val_loss: 0.6892
Epoch 5/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 0.6667 - loss: 0.6817 - val_accuracy: 0.5000 - val_loss: 0.6889


In [6]:
# --- 5. Export Options ---
# Save model
model.save('/content/my_sentiment_model.h5')

# Export predictions
predictions = (model.predict(X_test) > 0.5).astype("int32")
results = pd.DataFrame({"Text": tokenizer.sequences_to_texts(X_test), "Predicted": predictions.flatten()})
results.to_csv('/content/sentiment_predictions.csv', index=False)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step


In [7]:
# --- 6. Interactive Gradio Demo ---
def predict_sentiment(text):
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=10)
    prediction = model.predict(padded)[0][0]
    return "Positive 😀" if prediction > 0.5 else "Negative 😡"

demo = gr.Interface(fn=predict_sentiment,
                    inputs=gr.Textbox(lines=2, placeholder="Enter a sentence..."),
                    outputs="text",
                    title="Sentiment Analysis Demo",
                    description="Type a sentence and see if it's positive or negative.")

demo.launch(share=True)  # share=True gives you a public link


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6a0c8ff5c8380869fe.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
